In [2]:
def load_file(filename):
    with open(filename, 'r') as f:
        return f.read().strip().splitlines()

In [3]:
load_file("example.txt")

['$ cd /',
 '$ ls',
 'dir a',
 '14848514 b.txt',
 '8504156 c.dat',
 'dir d',
 '$ cd a',
 '$ ls',
 'dir e',
 '29116 f',
 '2557 g',
 '62596 h.lst',
 '$ cd e',
 '$ ls',
 '584 i',
 '$ cd ..',
 '$ cd ..',
 '$ cd d',
 '$ ls',
 '4060174 j',
 '8033020 d.log',
 '5626152 d.ext',
 '7214296 k']

In [4]:
def parse_line(line):
  if (line.find('$ ') == 0):
    rest = line[2:]
    if (rest.find('cd ') == 0):
      return dict(type="change_directory", variable=rest[3:])
    elif (rest.find('ls') == 0):
      return dict(type="list_directory")
    else:
      print("INVALID COMMAND", line)
  else:
    if (line.find('dir') == 0):
      [_, name] = line.split(' ')
      return dict(type="directory", name=name)
    else:
      [size, name] = line.split(' ')
      return dict(type="file", size=int(size), name=name)

In [5]:
[parse_line(line) for line in load_file("example.txt")]

[{'type': 'change_directory', 'variable': '/'},
 {'type': 'list_directory'},
 {'type': 'directory', 'name': 'a'},
 {'type': 'file', 'size': 14848514, 'name': 'b.txt'},
 {'type': 'file', 'size': 8504156, 'name': 'c.dat'},
 {'type': 'directory', 'name': 'd'},
 {'type': 'change_directory', 'variable': 'a'},
 {'type': 'list_directory'},
 {'type': 'directory', 'name': 'e'},
 {'type': 'file', 'size': 29116, 'name': 'f'},
 {'type': 'file', 'size': 2557, 'name': 'g'},
 {'type': 'file', 'size': 62596, 'name': 'h.lst'},
 {'type': 'change_directory', 'variable': 'e'},
 {'type': 'list_directory'},
 {'type': 'file', 'size': 584, 'name': 'i'},
 {'type': 'change_directory', 'variable': '..'},
 {'type': 'change_directory', 'variable': '..'},
 {'type': 'change_directory', 'variable': 'd'},
 {'type': 'list_directory'},
 {'type': 'file', 'size': 4060174, 'name': 'j'},
 {'type': 'file', 'size': 8033020, 'name': 'd.log'},
 {'type': 'file', 'size': 5626152, 'name': 'd.ext'},
 {'type': 'file', 'size': 721429

In [6]:
def create_tree(lines):
  root = dict(type="directory", name="root", children=[], parent=None)
  current = root
  for line in lines:
    parsed = parse_line(line)
    if (parsed["type"] == "change_directory"):
      if (parsed["variable"] == ".."):
        current = current["parent"]
      else:
        current = [child for child in current["children"] if child["name"] == parsed["variable"]][0]
    elif (parsed["type"] == "directory"):
      current["children"].append(dict(type="directory", name=parsed["name"], children=[], parent=current))
    elif (parsed["type"] == "file"):
      current["children"].append(parsed)
  return root

In [7]:
create_tree(load_file("example.txt")[1:])

{'type': 'directory',
 'name': 'root',
 'children': [{'type': 'directory',
   'name': 'a',
   'children': [{'type': 'directory',
     'name': 'e',
     'children': [{'type': 'file', 'size': 584, 'name': 'i'}],
     'parent': {...}},
    {'type': 'file', 'size': 29116, 'name': 'f'},
    {'type': 'file', 'size': 2557, 'name': 'g'},
    {'type': 'file', 'size': 62596, 'name': 'h.lst'}],
   'parent': {...}},
  {'type': 'file', 'size': 14848514, 'name': 'b.txt'},
  {'type': 'file', 'size': 8504156, 'name': 'c.dat'},
  {'type': 'directory',
   'name': 'd',
   'children': [{'type': 'file', 'size': 4060174, 'name': 'j'},
    {'type': 'file', 'size': 8033020, 'name': 'd.log'},
    {'type': 'file', 'size': 5626152, 'name': 'd.ext'},
    {'type': 'file', 'size': 7214296, 'name': 'k'}],
   'parent': {...}}],
 'parent': None}

In [13]:
def get_directory_size(directory):
  return sum([child["size"] for child in directory["children"] if child["type"] == "file"])

def get_sizes(directory):
  sizes = []
  indirect_size = 0
  for child in directory["children"]:
    if (child["type"] == "directory"):
      sizes += get_sizes(child)
      indirect_size += sizes[-1]["size"]
  size = get_directory_size(directory) + indirect_size
  sizes.append(dict(name=directory["name"], size=size))
  return sizes

In [14]:
get_sizes(create_tree(load_file("example.txt")[1:]))

[{'name': 'e', 'size': 584},
 {'name': 'a', 'size': 94853},
 {'name': 'd', 'size': 24933642},
 {'name': 'root', 'size': 48381165}]

In [15]:
dirs = [d for d in get_sizes(create_tree(load_file("input.txt")[1:])) if d["size"] <= 100000]
sum([d["size"] for d in dirs])

1845346

In [16]:
def which_to_delete(sizes):
  max_directory = max(sizes, key=lambda d: d["size"]) 
  total_space = 70000000
  needed_space = 30000000
  free_space = total_space - max_directory["size"]
  space_to_delete = needed_space - free_space
  print(free_space, space_to_delete)

  large_enough_dirs = [d for d in sizes if d["size"] >= space_to_delete]
  return min(large_enough_dirs, key=lambda d: d["size"])

In [17]:
which_to_delete(get_sizes(create_tree(load_file("input.txt")[1:])))

26370984 30000000


{'name': 'lfnhb', 'size': 3636703}